In [1]:
# 逻辑电路设置，X输入向量，1表示高电位 true， 0 表示低电位 false; W为调节阈值，通过不同的调试可以实现 与或非 和 与非门
# 这个页面中除了调试相关的方法，所有的逻辑运算（与、或、非、与非、异或）都是通过这个单一的逻辑单元电路实现的
# 实现加减乘除法 还需要以下几个功能
#   1， 用一组逻辑单元来表示一个二进制数，程序中用长度位16的数组来表示16位的二进制数
#   2， 需要实现移位操作，这个在电路中也是比较容易实现的
#   3， 重复操作，对应的语句 for range(16) ，实际的电路就是要一个 16组的逻辑门
#   4， 计算机还需要一个条件跳转指令，goto when 0/1 这个暂时没有用到
def logicalGate(X, W):
    return 1 if X[0]*W[0] + X[1]*W[1] + X[2]*W[2] > 0 else 0

In [2]:
# 与门 输入电位 减半叠加 和 0.75相比来判断是否为 高低电位
# 输入的取值 只能是 0 或者 1 ，返回结果也只是 0 或者 1
def AND(x1, x2):
    return logicalGate([1,x1,x2], [-0.75, 0.5, 0.5])

In [3]:
# 或门 输入电位 减半叠加 和 0.25 相比
def OR(x1, x2):
    return logicalGate([1,x1,x2], [-0.25, 0.5, 0.5])

In [4]:
# 非门 输入电位 反向减半加倍 和 -0.75 相比
def NOT(x1):
    return logicalGate([1,x1,x1], [0.75, -0.5, -0.5])

In [5]:
# 与非门 非门 输入电位 反向减半叠加 和 -0.75 相比
def NAND(x1, x2):
    return logicalGate([1,x1,x2], [0.75, -0.5, -0.5])

In [6]:
# 异或门 无法直接用单个感知器来实现，
def XOR(x1, x2):
    return AND(OR(x1,x2),NAND(x1,x2))

In [7]:
#测试单目运算
NOT(0),NOT(1)

(1, 0)

In [8]:
# 测试其他的逻辑运算
def testGate(gates) :
    ret = []
    for x1 in range(2):
        for x2 in range(2): 
            item = []
            for gate in gates :
                item.append(gate(x1,x2))
            ret.append(item)
    return ret

In [9]:
# 显示测试结果
def printTestRet(ret):
    print ( 'x1\tx2\tand\tor\tnand\txor' )
    for x1 in range(2):
        for x2 in range(2):
            print(x1, '\t', x2, end ='\t')
            for i in range(4):
                print(ret[x1*2+x2][i], end = '\t' )
            print()

In [10]:
ret = testGate([AND,OR,NAND,XOR])
printTestRet(ret)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [11]:
# 用与非门 实现 与门
def AND2(x1, x2):
    return NAND(NAND(x1,x2), NAND(x1,x2) )

In [12]:
# 用与非门 实现 或门
def OR2(x1, x2):
    return NAND(NAND(x1,x1), NAND(x2,x2) )

In [13]:
# 用与非门 实现 非门
def NOT2(x1):
    return NAND(x1,x1)

In [14]:
# 用与非门 实现 与非门 ，就是本身
def NAND2(x1, x2):
    return NAND(x1, x2 )

In [15]:
# 用与非门 实现 异或门
def XOR2(x1, x2):
    return NAND(NAND(x1, NAND(x2,x2) ), NAND(NAND(x1,x1),x2) )

In [16]:
NOT2(0),NOT2(1)

(1, 0)

In [17]:
ret2 = testGate([AND2,OR2,NAND2,XOR2])
printTestRet(ret2)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [18]:
def XOR1(x1, x2):
    return AND(OR(x1,x2),NOT(AND(x1,x2)))

In [19]:
for xs in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    y = XOR1(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

(0, 0) -> 0
(1, 0) -> 1
(0, 1) -> 1
(1, 1) -> 0


In [20]:
# 半加器的实现
# 返回 结果 和 进位
def halfAdd(x1, x2):
    return (XOR(x1, x2), AND(x1, x2))

In [21]:
# 全加器，只能实现一位二进制相加，x1, x2是两个加数， carryBit 进位
# 返回 结果 和 进位
def fullAdd(x1, x2, carryBit):
    s, co = halfAdd(x1, x2)
    s, co2 = halfAdd(carryBit, s)
    return (s, OR(co, co2))

In [22]:
for xs in [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]:
    y = fullAdd(xs[0], xs[1], xs[2])
    print(str(xs) + " -> " + str(y))

(0, 0, 0) -> (0, 0)
(1, 0, 0) -> (1, 0)
(0, 1, 0) -> (1, 0)
(1, 1, 0) -> (0, 1)
(0, 0, 1) -> (1, 0)
(1, 0, 1) -> (0, 1)
(0, 1, 1) -> (0, 1)
(1, 1, 1) -> (1, 1)


In [23]:
# 16位二进制 0
def ZERO16BIT():
    v16bit =[]
    for i in range(16):
        v16bit.append(0)
    return v16bit

In [24]:
# 16位二进制相加， co 为是否进位标志
# 最高位15位为正负号 负数
# 返回结果 
def add16Bit(a16bit, b16bit, co=0):
    v16bit = ZERO16BIT()
    for i in range(16):
        (d, co) = fullAdd(a16bit[i], b16bit[i], co)
        v16bit[i] = d
    # co>0 溢出
    return v16bit

In [25]:
# 整数编码器 辅助方法，用数组来表示 16位数
def to16bit(x):
    (sign, d) = (1, x+65536) if x<0 else (0, x)
    # d = x
    v16bit =ZERO16BIT()
    for i in range(16):
        v16bit[i] = d % 2
        d = d // 2
    v16bit[15] = sign
    return v16bit

In [26]:
# 整数解码器 辅助方法，将16位数数组 转换为整数
def from16bit(a16bit):
    d=0
    for i in range(15,-1,-1):
        d = d * 2 + a16bit[i]
    return (d-65536) if a16bit[15] > 0  else d
        

In [27]:
# 加法的实现
def add(a, b):     
    return from16bit(add16Bit(to16bit(a), to16bit(b)))

In [28]:
add(6,8), add(6,-7), add(9,-7), add(-6,-7)

(14, -1, 2, -13)

In [29]:
#溢出测试
add(65536, 0), add(65536, 65535), add(0, 65535), add(65530, 65530), add(32780, 32780)

(0, 32767, 32767, -12, 24)

In [30]:
# 16位数的补码，就是每一位取反， 但是最低为没有+1
def complement(a16bit):
    v16bit = ZERO16BIT()
    for i in range(16):
        v16bit[i] = NOT(a16bit[i])
    return v16bit

In [31]:
# 16位数的减法实现
def sub16Bit(a16bit, b16bit): 
    return add16Bit(a16bit, complement(b16bit), 1)

In [32]:
def sub(a, b):
    return from16bit(sub16Bit(to16bit(a), to16bit(b)))

In [33]:
sub(32, 19), sub(12, 32), sub(12, -10), sub(-12, -32), sub(-42, -22)

(13, -20, 22, 20, -20)

In [34]:
# 模拟 左移位操作
def shitRight(a16bit, shiftBits = 1):
    v16bit = ZERO16BIT()
    for i in range(shiftBits, 16):
        v16bit[i] = a16bit[i -shiftBits]
    return v16bit

In [35]:
# 模拟 左移位操作
def shiftLeft(a16bit, shiftBits = 1):
    v16bit = ZERO16BIT()
    for i in range(16 - shiftBits):
        v16bit[i] = a16bit[i+shiftBits]
    return v16bit

In [36]:
from16bit(shiftLeft(to16bit(12))), from16bit(shitRight(to16bit(12)))

(6, 24)

In [37]:
# 逐位与 可以用 if 语句代替 a16bit if x>0 else 0
def AND16BIT1BIT(a16bit, x):
    v16bit = ZERO16BIT()
    for i in range(16):
       v16bit[i] = AND(a16bit[i], x)
    return v16bit

In [38]:
# 16位数的乘法实现
# 不考虑溢出情况
def multi16Bit(a16bit, b16bit): 
    v16bit = ZERO16BIT()
    for i in range(16):
        v16bit = add16Bit(v16bit, AND16BIT1BIT(a16bit, b16bit[i]))
        a16bit = shitRight(a16bit)
    return v16bit

In [39]:
def multi(a, b):
    return from16bit(multi16Bit(to16bit(a), to16bit(b)))

In [40]:
multi(7, 9), multi(11, -5), multi(-7, 17), multi(-17, -19)

(63, -55, -119, 323)

In [41]:
#溢出测试
multi(16384, 16384), multi(255, 255), multi(337, 493), multi(1024, 255), 

(0, -511, -30467, -1024)

In [42]:
# 检查正负号，最高位 1 表示负，最高位 0 表示正
# 返回 1 表示正 0 表示 负
def checkSign(a16bit):
    return NOT(a16bit[15])

In [43]:
# 比较大小 1 a>=b, 0 a<b
checkSign(sub16Bit(to16bit(5), to16bit(7)))

0

In [44]:
# 求16位数的相反数
def oppositeNumber(a16bit):
    return add16Bit(complement(a16bit), to16bit(1))


In [45]:
from16bit(oppositeNumber(to16bit(-5))), from16bit(oppositeNumber(to16bit(6)))

(5, -6)

In [46]:
# 实现 a16bit if cond else b16bit
def NVL(cond, a16bit, b16bit):
    return  add16Bit(AND16BIT1BIT(a16bit, cond), AND16BIT1BIT(b16bit, NOT(cond)))

In [47]:
# 16位数 正整数的除法实现
# 返回 结果和余数
def div16BitPositive(a16bit, b16bit): 
    div16bit = ZERO16BIT() # 被除数，也是最后的余数
    v16bit = ZERO16BIT()
    for i in range(15, -1, -1):
        div16bit = shitRight(div16bit)
        v16bit = shitRight(v16bit)
        div16bit[0] = a16bit[i]
        stepV = sub16Bit(div16bit, b16bit)
        v16bit[0] = checkSign(stepV)
        # print (from16bit(div16bit), from16bit(v16bit), from16bit(stepV) ,  v16bit[0])
        # 下面这行可以用一个 if语句
        # div16bit = stepV if v16bit[0] > 0 else div16bit
        # add16Bit(AND16BIT1BIT(div16bit, NOT(v16bit[0])), AND16BIT1BIT(stepV, v16bit[0]))
        div16bit = NVL(v16bit[0], stepV, div16bit)
    return (v16bit,div16bit)

In [48]:
# 16位数的除法实现
# 除法需要考虑正负号的影响
# 返回 结果和余数
def div16Bit(a16bit, b16bit): 
    aSign = checkSign(a16bit)
    bSign = checkSign(b16bit)
    # 取绝对值
    va16bit = NVL(aSign, a16bit,  oppositeNumber(a16bit)) 
    vb16bit = NVL(bSign, b16bit,  oppositeNumber(b16bit))
    (c, d) = div16BitPositive(va16bit, vb16bit)
    c = NVL(XOR(aSign, bSign), oppositeNumber(c), c)
    d = NVL(aSign, d, oppositeNumber(d))
    return (c,d)

In [49]:
def div(a, b):
    (c,d) = (div16Bit(to16bit(a), to16bit(b)))
    return (from16bit(c), from16bit(d))

In [50]:
div(126, 32), div(126, -32), div(-126, 32), div(-126, -32)

((3, 30), (-3, 30), (-3, -30), (3, -30))

In [51]:
#除0测试
div(1, 0), div(0, 0), div(-7, 0), div(139, 0)

((-1, 1), (-1, 0), (1, -7), (-1, 139))